# Improving and combining functions

## Outline

- Curried functions
	- What are Curried functions?
    - Partial application and simple example
	- Useful example
<br><br>
- Higher-order functions
	- What are they?
	- Why are they useful?
	- Example taking a function as a parameter
	- Example returning a function as a result
<br><br>
- Applying and composing functions 
	- precedence, and associativity
	- Function composition
	- The `$` operator
	- Composition and the `.` operator
	- Point-free style
<br><br>
- Lambda functions
	- What are they?
	- Why do I care?
	- How to use them
	- Other examples

## Curried functions

### What are curried functions?

In mathematics and computer science, currying is the technique of converting a function that takes multiple arguments into a sequence of functions that each takes a single argument. 

The converted function is then called a **curried function**. It is named after the American mathematician and logician **Haskell Curry**.

Under the hood, every function in Haskell officially only takes one parameter. All the functions that accepted several parameters so far have been **curried functions**.

So how is it possible that in Haskell there are functions that take in more than one parameter? To understand how this works in Haskell we have to understand **partial application**.

### Partial application

Partial application in Haskell means that you provide less arguments to a function then it accepts.

The result you get is a new function which takes in the rest of the parameters that you did not provide to the original function. 

Let's understand it on an example of the `max` function: `max :: a -> a -> a`. It takes in two parameters and returns the larger one.

`a` is called a type parameter and can be of any type that supports comparison operations, as for instance `Int`, `Double`, `Char`.

You can apply the function to only one parameter and you get another function that takes in only one parameter:

In [ ]:
fourOrLarger :: Int -> Int
fourOrLarger = max 4

fourOrLarger' :: Int -> Int
fourOrLarger' x = max 4 x

fourOrLarger 3
fourOrLarger 5

Notice that even though we specified the type signature as `Int -> Int` we did not provide any input argument to `fourOrLarger` as in `fourOrLarger'`.

When we say `max 4` we use partial application and Haskell created a new function that accepts only one argument and compares it to 4 with the `max` function.

The type of the `max` function can also be written as `max :: a -> (a -> a)`. You can read it as: the `max` function takes an `a` and returns a function that takes an `a` and returns an `a`.

That's why the return type and the parameters of functions are all simply separated with arrows.

The first version `max :: a -> a -> a` is an **uncurried function**, while the second one `max :: a -> (a -> a)` is **curried**.

### Useful example

Lets say you have a function that is used to create an email in the format `name.lastName@domain`. The parameters you provide it are the domain, the name and the last name.

In [ ]:
createEmail :: String -> String -> String -> String
createEmail domain name lastName = name ++ "." ++ lastName ++ "@" ++ domain

Now your company has only two other companies as clients which have two different domain names. You do not want your users to write out the domain name every time.

So you create 2 functions where you provide their domain names to the `createEmail` function.

In [ ]:
createEmailTweag :: String -> String -> String
createEmailTweag = createEmail "tweag.com"

createEmailWellTyped :: String -> String -> String
createEmailWellTyped = createEmail "welltyped.com"

createEmailTweag "Shanai" "Twain"
createEmailWellTyped "Bob" "Dylan"

It's important to notice that this is only possible because in the function `createEmail` we set the domain parameter as the first one. 

So the order of the arguments matters and should be preferably from most general to leas general.

If for some reason the parameter you want to apply is not first and you are not allowed to rewrite the exisitng function you can make a helper function.

In [ ]:
swap1and3 :: (a -> a -> a) -> a -> a -> a -> a
swap1and3 f x1 x2 x3 = f x3 x2 x1

createEmail' = swap1and3 createEmail
createEmail' "Twain" "Shanai" "tweag.com"

## Higher-order functions

### What are they

As we've already learned, functions are data in Haskell. You can pass them them as a input parameter to another function or return it as a result.

Because we can do this in Haskell we say that functions are **first class citizens**.

The function that operates on other functions as arguments and / or returns a function as a result is called a **higher-order function**.

### Why are they useful

They are useful because it gives you more flexibily to design code if your functions can also take in other functions as parameters or return them as a result.

Haskell has some build in functions that are higher order functions which are commonly used for working with lists. We will learn about them in the Lists lesson.

In fact using functions as arguments is generaly an extremely common practice in Haskell.

### Example taking a function as parameter

Lets create a simple function and then another function that takes the first one as a parameter.

In [ ]:
add :: Int -> Int -> Int
add x y = x + y

applyToThree :: (Int -> Int -> Int) -> Int -> Int -> Int -> Int
applyToThree f x1 x2 x3 = x1 `f` x2 `f` x3

applyToThree add 1 2 3

This gets usefull if we have a complex long function `complexAndLong` instead of `applyToThree` in our code. 

And somewhere in this function we need to apply another function that can change, depending on what we want to do.

So instead of hardcoding the external functions and making several versions of `complexAndLong` we can make it a higher-order function.

### Example returning a function as a result

We have already seen an example of that when we wrote the functions `fourOrLarger` and `swap1and3`.

The first one does not take in any functions and returns a function. And it uses partial application on a Haskell build-in function.

The second one takes in a function and also returns a function. It does not use partial application.

## Applying and composing functions 

### Precedence and associativity

Precedence means the priority an operator has. So if we use two operators with different precedence the one with the higher precedence gets applied first.

We can get the precedence for an operator with the info command `:i`. 

In [ ]:
:i (+)
-- infixl 6 +
:i (*)
-- infixl 7 *

1 + 2 * 3

Because multiplication has a precedence of 7 which is higher then the addition precedence 6, ther result is 7 and not 9.

We see also the `:i` command returns the keyword `infixl` which means the operators are infix operators and `l` stands for left associative.

Together the informatin `infixl 6 +` is called **fixity**.

Associativity for an operator tells you if you apply this operator twice which side (left or right) will be evaluated first.

For the operators `(+)` and `(\*)` the expressions below evaluate the left side first (`1 + 2` and `1 * 2` get evaluated first).

In [ ]:
1 + 2 + 3
1 * 2 * 3

In the above example the associativity does not matter. But for the exponentiation operator `(**)` which is right associative it does matter.

In [ ]:
2**3**4
(2**3)**4

We see if it would be left associative the result would be different. When we create our own operator we can also define the precedence and associativity.

In [ ]:
x +++ y = x + y; infixl 7 +++

1 +++ 2 * 3

Now the result is 9, because `+++` and `*` are both left associative and have the same precedence.

But we get a compile error if we use operators of different associativity and same precedence.

In [ ]:
x +++ y = x + y; infixr 7 +++

1 +++ 2 * 3

### Function composition

Recall the function `checkLocalHost` from a previous lesson that checks if the argument is a localhost or not and reports the user the result.

In [ ]:
checkLocalhost :: String -> String
checkLocalhost ip =
    if ip == "127.0.0.1"
        then "It’s a localhost!"
        else "No, it's not a localhost."

A function `fourToIP` takes four integers and makes IP address. Then we can use it together with the previous function.

In [ ]:
fourToIP :: Int -> Int -> Int -> Int -> String
fourToIP a b c d = show a ++ "." ++ show b ++ "." ++ show c ++ "." ++ show d

checkLocalhost (fourToIP 127 0 0 1)

Two functions are **composed** here, `checkLocalhost` and `fourToIP` because `fourToIP` returns `String` and `checkLocalhost` function has `String` at the input. 

Schematically, this can be represented as follows:

$$
    \boxed{\mathrm{(Int, Int, Int, Int)}}
        \xrightarrow{\mathrm{~~~~~~fourToIP~~~~~~}}
    \boxed{\mathrm{String}}
        \xrightarrow{~~~~~~\mathrm{checkLocalhost}~~~~~~}
    \boxed{\mathrm{String}}
$$

The way of composing functions above uses parenthesis. But what will happen if we need to compose three, four or more functions? 

The number of parenthesis grows, and our code becomes less readable. There is another way how to compose functions.

### The application `$` operator

The application operator has the following type signature and fixity:
```haskell
:i ($)
($) :: (a -> b) -> a -> b
infixr 0 $
```

We see it takes in a function and a variable and then applies the function to it. It is right associative and it has a precedence of 0. 

Because of that any expression you write on the right side will be evaluated first and then the function will be applied to the result.

In [ ]:
(2 *) 3 + 4
(2 *) $ 3 + 4

max 5 4 + 2
max 5 $ 4 + 2

We see how the application operator changes the result of the above expressions. We can also chain multiple function calls with it.

In [ ]:
show ((2**) (max 3 (2 + 2)))
show $ (2**) $ max 3 $ 2 + 2

So for the example in the previous chapter we could now re-write it to:

In [ ]:
checkLocalhost $ fourToIP 127 0 0 1

In general you can re-write chained functions calls that use parantheses always with the application operator:
```haskell
f1 (f2 (f3 (... x)))
or
f1 $ f2 $ f3 $ ... x
```

### Composition and the dot `.` operator

Another way to compose functions is with the dot operator. It has the following type signature and fixity:
```haskell
:i (.)
(.) :: (b -> c) -> (a -> b) -> a -> c
infixr 9 .
```

It takes in two functions and a parameter and applies the functions to the parameter. Here is an example.

In [ ]:
add1, add2 :: Int -> Int
add1 x = x + 1
add2 x = x + 2

(add2 . add1) 3
add2 . add1 $ 3

add3 :: Int -> Int
add3 x = add1 . add2 $ x
add3 3

We see that we need to use parentheses around the composed functions when applying them to a variable.

Also the application operator can be used. Or we can just declare a new function by composing the first two.

If we rewrite now an example from the application operator chapter by using the dot operator we get:

In [ ]:
show ((2**) (max 3 (2 + 2)))
show . (2**) . max 3 $ 2 + 2

Again we can rewrite our initial example as follows:

In [ ]:
(checkLocalhost . fourToIP) 127 0 0 1

Instead of two functions `checkLocalhost` and `fourToIP` we obtain a new one `checkLocalhost . fourToIP` called their **composition**. Schematically, this can be represented as follows:

$$
    \boxed{\mathrm{(Int, Int, Int, Int)}}
        \xrightarrow{\mathrm{~~~~~~fourToIP~~~~~~}}
    \boxed{\mathrm{String}}
        \xrightarrow{~~~~~~\mathrm{checkLocalhost}~~~~~~}
    \boxed{\mathrm{String}}
    ~~~~~
    =
    ~~~~~
    \boxed{\mathrm{(Int, Int, Int, Int)}}
        \xrightarrow{\mathrm{~~~~~~checkLocalhost~.~fourToIP~~~~~~}}
    \boxed{\mathrm{String}}
$$


### Point-free style

This style in Haskell means we are declaring functions without writing out the variables.

We saw allready one example when we used partial application. Below you can see the some of the previous defined functions in point-free style.

In [ ]:
fourOrLarger :: Int -> Int
fourOrLarger = max 4

add3 :: Int -> Int
add3 = add1 . add2

composition :: Int -> Int -> Int -> Int -> String
composition = checkLocalhost . fourToIP

## Lambda functions

The term lambda function comes from the mathematical system called **lambda calculus** developed in the 1930s by Alonzo Church. 

Since this system can simulate a Turing machine it can be used for programming and Haskell is based on lambda calculus.

### What are they?

The idea of ​the ​lambda functions or so-called anonymous function is that you can define it without naming it.

Here's how a multiplication lambda function $f(x,y)=x*y$ looks like in Haskell:

```haskell
\x y -> x * y
```
The backslash `\` at the beginning denotes the beginning of the lambda function, after which the arguments follow. And after the arrow `->` comes the main body of the function.

### Why do I care?

Sometimes we don't want to write functions out and we rather define them directly. It can make the code more compact.

You will also see later in the Monad lesson that lambda functions are used by Haskell internally when it does some code syntax translation in the background. 

### How to use them

An ordinary function starts with a signature/definition, and it is used somewhere by its name.

A lambda function doesn't need to have a signature and a name, and it can be applied immediately.

Here is an example:

In [ ]:
(\x y -> x * y) 2 3

Any function has a type, so does a lambda function. It is a polymorphic function, meaning the types are not fixed but are type variables. 

In the previous example, the `mult` has the following type signature show below. This means that the input parameters can be of any type for which the multiplication operation is defined.
```haskell
mult :: a -> a -> a
```

Same for the lambda function. You can input data to the function which are of types for which the operators in the lambda function can operate on them.

Lambda functions can also be given *names*, and we can call them as ordinary functions (for instance, using `where` construction).

In [ ]:
rectangleArea :: Double -> Double -> Double
rectangleArea width length = mult width length
    where mult = \x y -> x * y

### Lexical scope

Lexical scope is a type of variable lookup that is used by Haskell. The priorities for lookup are following: 

* when you declare variable names in lambda functions they have the highest priority

* next priority have function varaiable names that are fdefined in the begining of the function

* the lowest priority have global variables names

Here is an example where you can see how these rules apply.

In [ ]:
x = 1 :: Int

add1, add2, add3 :: Int -> Int
add1 y = x + y
add2 y = (\x -> x + y) 2
add3 y = (\y -> (\x -> x + y) 4) 3

### Other examples

You can also chain lambda functions one inside the other. Here is an example:

In [ ]:
add3 :: Int -> Int -> Int -> Int
add3 = \x -> \y -> \z -> x + y + z

Or you can define a function as ussually and then use also a lamdba function inside of it.

In [ ]:
mixedFunction :: Int -> Int -> String -> String
mixedFunction x y = \z -> show (x + y) ++ " and " ++ z

mixedFunction 1 2 "3"

You can see that the first parameters for the input are the ones defined in the function and the last one is the lambda function parameter.

## Summary

In this lesson, we've discussed:

* How to curry functions and how does partial application work.

* What are higher-order functions and why are they useful.

* Various ways of composing and applying functions.

* Lambda functions and how to use them.